In [1]:
import warnings
warnings.simplefilter('ignore')
import gc
import re
import time
import os
from sklearn.model_selection import KFold, train_test_split, GridSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn import metrics
from scipy.stats import entropy, kurtosis
import xgboost as xgb
import seaborn as sns
from xgboost import plot_importance
import matplotlib.pyplot as plt
import pandas as pd
from math import *
from sklearn import datasets
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import GroupKFold
from sklearn.datasets.samples_generator import make_blobs
import numpy as np
import lightgbm as lgb
from IPython.display import display
import keras
from keras import backend as K
from keras.models import Model
from keras.layers import multiply
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import log_loss
from keras.layers import Dense, Embedding, Input, Flatten, Masking, Activation
from keras.layers import LSTM, Bidirectional, Dropout
from keras.layers import Conv2D, MaxPooling2D, GlobalMaxPooling2D 
from keras.layers import Conv1D, MaxPooling1D, GlobalMaxPooling1D 
from keras.preprocessing import text, sequence
from keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard
from keras.layers.wrappers import TimeDistributed
from sklearn.metrics import roc_auc_score, accuracy_score
import tensorflow as tf
from sklearn.model_selection import KFold
import time  # pd.set_option('display.max_columns', None)
# pd.set_option('max_columns', None)
# pd.set_option('max_rows', None)
pd.set_option('float_format', lambda x: '%.6f' % x)

%matplotlib inline

Using TensorFlow backend.


In [2]:
pathf = os.path.join("..", "data", "particles")

trainpd = pd.read_csv(os.path.join(pathf, "train.csv"))
print(trainpd.head(1))
trainshape = trainpd.shape
print(trainshape)
eventpd = pd.read_csv(os.path.join(pathf, "event.csv"))
print(eventpd.head(1))
print(eventpd.shape)
testpd = pd.read_csv(os.path.join(pathf, "test.csv"))
testshape = testpd.shape
print(testpd.head(1))
print(testpd.shape)

            x           y  z          t   terror        q  flag  event_id  \
0 -142.500000 -147.500000  0 767.879000 2.029660 1.050520     0         7   

   hit_id  
0       1  
(9473201, 9)
   event_id  nhit  nhitreal     energymc   thetamc     phimc       xcmc  \
0         7   426        70 48348.900000 63.168600 11.098200 -40.830000   

        ycmc  
0 114.030000  
(13315, 8)
            x           y  z          t   terror        q  event_id  hit_id
0 -142.500000 -127.500000  0 848.061000 1.998400 1.150670         9       1
(4086511, 8)


In [3]:
# (k(q,mc)*(t0+L))^2 + dis^2 -dis*cos(phi)*sin(thmc)*(t0+L) = (t+L)^2
# t0 方程 
# a = k(q,mc)^2
# b = 2*L*k(q,mc)^2 -dis*cos(phi)*sin(thmc)
# c = L^2 * k(q,mc)^2 + dis^2 - dis*cos(phi)*sin(thmc)*L - (t+L)^2 
# t0 = (-b +- (b^2 - 4*a*c)^(1/2))/2*a
data = pd.concat([trainpd, testpd], ignore_index=True)
data = pd.merge(data, eventpd, on='event_id', how='left')

data['fx'] = data['x'] - data['xcmc']
data['fy'] = data['y'] - data['ycmc']
data['phimc'] = data['phimc'] * np.pi / 180.
data['fphi'] = np.arctan2(data['fy'], data['fx']) - data['phimc']
data['fdis'] = np.sqrt(data['fx'] ** 2 + data['fy'] ** 2)
data['thetamc'] = data['thetamc'] * np.pi / 180.

data['fsinthmc'] = np.sin(data['thetamc'])
data['fsinthmc_v'] = 1.0/data['fsinthmc']
data['fcosphi'] = np.cos(data['fphi'])
data['fcosphi_v'] = 1.0/data['fcosphi']

data['fcosthmc'] = np.cos(data['thetamc'])
data['fcosthmc_v'] = 1.0/data['fcosthmc']
data['fsinphi'] = np.sin(data['fphi'])
data['fsinphi_v'] = 1.0/data['fsinphi']

data['ftanphi'] = np.tan(data['fphi'])
data['ftanphi_v'] = 1.0/data['ftanphi']
data['ftanthmc'] = np.tan(data['thetamc'])
data['ftanthmc_v'] = 1.0/data['ftanthmc']


# data['ft2'] = data['t'] ** 2
# data['fdis2'] = data['fdis'] ** 2

data['fttrue'] = data['t'] / data['terror']
data['terror_v'] = 1.0 / data['terror']
data['terror_v2'] =data['terror_v'] ** 2 
data['fttrue_v'] = 1.0 / data['fttrue']
data['fttrue2'] = data['fttrue'] ** 2
data['fttrue2_v'] = 1.0 / data['fttrue2'] 
data['nhitratio'] = data['nhit'] / data['nhitreal']
data['nhitratio_v'] = data['nhitratio']
data['energymc_v'] = 1.0 / data['energymc']
data['fenergymc2'] = data['energymc'] ** 2
data['fenergymc2_v'] = 1.0 / data['fenergymc2'] 
# data['q_v'] = 1.0 / data['q']
data['q2'] = data['q']
# data['q2_v'] = 1.0 / data['q2']

del data['fx']
del data['fy']
del data['x']
del data['y']
del data['z']
del data['xcmc']
del data['ycmc']
del data['fphi']
del data['phimc']
del data['nhitreal']
gc.collect()

11

In [4]:
info_new = pd.DataFrame()
info_new["event_id"] = data.groupby(["event_id"])["event_id"].mean()
info_new["fdis_mean"] = data.groupby(["event_id"])["fdis"].mean()
info_new["fdis_std"] = data.groupby(["event_id"])["fdis"].std()
info_new["fdis_stdmean"] = info_new["fdis_std"] / info_new["fdis_mean"]
info_new["ft_min"] = data.groupby(["event_id"])["t"].min()
info_new["ft_max"] = data.groupby(["event_id"])["t"].max()
info_new["t_mean"] = data.groupby(["event_id"])["t"].mean()
info_new["ft_std"] = data.groupby(["event_id"])["t"].std()
info_new["ft_stdmean"] = info_new["ft_std"] / info_new["t_mean"]
info_new["ft_mean"] = (info_new['t_mean']-info_new['ft_min']) / (info_new['ft_max']-info_new['ft_min'])
info_new.reset_index(drop=True, inplace=True)
data = pd.merge(data, info_new, on='event_id', how='left')

# data['ft_rel'] = (data['t']-data['ft_min']) / (data['ft_max']-data['ft_min'])
data['ft_rel'] = data['t'] / data['ft_std']
# data['ft_rel_std'] = data['ft_rel'] / data['ft_std']
data['ft2_rel'] = data['ft_rel'] ** 2
# data['ft2_rel_std'] = data['ft_rel_std'] ** 2
data['ft_rel_v'] = 1.0 / data['ft_rel']
data['ft2_rel_v'] = 1.0 / data['ft2_rel'] 
# (k(q,mc)*(t0+L))^2 + dis^2 -dis*cos(phi)*sin(thmc)*(t0+L) = (t+L)^2
data = data.sort_values(by=['event_id', 'ft_rel']).reset_index(drop=True)
for i in [1]:
# for i in [4, 6, 8, 10, 12]:
# for i in [7, 8, 11,17, 47]:
    data[f'ft_{i}diff'] = data.groupby('event_id')['ft_rel'].diff(periods=i).fillna(0)
    
data['fdis_rel'] = data['fdis'] / data['fdis_mean']
data['fdis_rel_std'] = data['fdis_rel'] / data['fdis_std']
data['fdis2_rel'] = data['fdis_rel'] ** 2
data['fdis2_rel_std'] = data['fdis_rel_std'] ** 2

data['fdis_t_rel'] = data['fdis_rel'] / data['ft_rel']
data['fdis_tmean_rel'] = data['fdis_rel'] / data['ft_mean']
data['fdis_t_rel'].fillna(0, inplace=True)
# data['fdis-t_rel'] = data['fdis_rel'] * data['ft_rel']
# data['fdis-tmean_rel'] = data['fdis_rel'] * data['ft_mean']
# data['fdis_t_normal'] = data['fdis_rel'] / data['ft_normal']
# data['fdis_t_normal'].fillna(0, inplace=True)
# data['fdis-t_normal'] = data['fdis_rel'] * data['ft_normal']

# data = data.sort_values(by=['event_id', 'fdis_rel']).reset_index(drop=True)
# for i in [5, 6, 7, 8, 10, 11,12]:
#     data[f'fdis_rel_{i}diff'] = data.groupby('event_id')['fdis_rel'].diff(periods=i).fillna(0)
data['fx_normal'] = data['fcosphi'] * data['fdis_rel']
# data['fy_normal'] = data['fsinphi'] * data['fdis_rel']

data['fcossin'] = data['fcosphi'] * data['fsinthmc']
data['fdis_relcossin'] =  data['fdis_rel'] * data['fcossin']
data['fdis2_relcossin'] = data['fdis2_rel'] * data['fcossin']
# data['fdis2_rel_stdcossin'] = data['fdis2_rel_std'] * data['fcossin']
data['ft2_relcossin'] = data['ft2_rel'] * data['fcossin']

data['fsincos'] = data['fsinphi'] * data['fcosthmc']
# data['fdis_relsincos'] = data['fdis_rel'] * data['fsincos']
# data['fdis2_relsincos'] = data['fdis2_rel'] * data['fsincos']
# data['fdis2_rel_stdsincos'] = data['fdis2_rel_std'] * data['fsincos']
# data['ft_relsincos'] =  data['ft_rel'] * data['fsincos']
# data['ft2_relsincos'] = data['ft2_rel'] * data['fsincos']

data['fsinsin'] = data['fsinphi'] * data['fsinthmc']
# data['fdis_relsinsin'] =  data['fdis_rel'] * data['fsinsin']
# data['fdis2_relsinsin'] = data['fdis2_rel'] * data['fsinsin']
# data['fdis2_rel_stdsinsin'] = data['fdis2_rel_std'] * data['fsinsin']
# data['ft_relsinsin'] =  data['ft_rel'] * data['fsinsin']
# data['ft2_relsinsin'] = data['ft2_rel'] * data['fsinsin']

data['fcoscos'] = data['fcosphi'] * data['fcosthmc']

data['ft_relsinsin'] = data['ft_rel'] * data['fsinsin']
data['ft_relcoscos'] = data['ft_rel'] * data['fcoscos']
data['ft_relsincos'] = data['ft_rel'] * data['fsincos']
data['ft_relcossin'] = data['ft_rel'] * data['fcossin']

data['ft_rel_errcoscos'] = data['ft_relcoscos'] * data['terror']
data['fdis2_rel_stdcoscos'] = data['fdis2_rel_std'] * data['fcoscos']
data['ft2_relcoscos'] = data['ft2_rel'] * data['fcoscos']

data['fdis_t_relcossin'] = data['fdis_relcossin'] / data['ft_rel']
data['fdis_t_relcossin'].fillna(0, inplace=True)
data['fdis-t_relcossin'] = data['fdis_relcossin'] * data['ft_rel']

# # 要保留
# del data['ft_7diff']
# del data['ft_8diff']
# del data['ft_11diff']
# del data['ft_17diff']
# del data['fdis_t_rel']
# del data['fttrue2']
# del data['ft_rel_errcoscos']
# del data['fdis_t_relcossin']
# del data['fdis-t_relcossin']
# del data['q2']
# del data['q']
# del data['ft_47diff']
# del data['ft2_rel_v']
# del data['fttrue2_v']
# del data['fttrue_v']
# del data['fsinthmc_v']
# del data['ft_rel']

# del data['t']
# del data['ft_relcossin']
# del data['ft2_rel']
# del data['ft_relcoscos']
# del data['ft_rel_v']
# del data['fcosphi_v']
# del data['ft2_relcossin']
# del data['ft2_relcoscos']
# del data['ftanphi']
# del data['ft_relsincos']
# del data['ft_relsinsin']
# del data['fttrue']
# del data['ftanphi_v']
# del data['fsinphi_v']

# 要保留但不明显
# del data['fdis']
# del data['fdis2_rel']
# del data['fdis_tmean_rel']
# del data['terror_v2']
# del data['nhitratio']
# del data['fdis2_relcossin']
# del data['fdis2_rel_stdcoscos']
# del data['ftanthmc']
# del data['energymc_v']
# del data['nhitratio_v']
# del data['fdis_rel_std']
# del data['fenergymc2']
# del data['fdis2_rel_std']
# del data['fdis_stdmean']

# 真删除
del data['fx_normal']
del data['fcossin']
del data['fdis_relcossin']
del data['terror_v']
del data['ft_std']
del data['fdis_rel']
del data['ftanthmc_v']
del data['fenergymc2_v']
del data['fcoscos']
del data['fcosthmc_v']
del data['fdis_mean']
del data['ft_mean']
del data['nhit']
del data['fdis_std']
del data['t_mean']
del data['energymc']
del data['terror']
del data['ft_stdmean']
del data['ft_min']
del data['ft_max']
del data['fcosphi']
del data['thetamc']
del data['fsinthmc']
del data['fcosthmc']
del data['fsincos']
del data['fsinsin']
del data['fsinphi']

gc.collect()

0

In [5]:
print(trainshape[0])
print(data.shape)

# print(data[data['hit_id']<0])
scaler = MinMaxScaler(feature_range=(0, 1))
# feature = [x for x in data.columns if x not in ['flag', 'index', 'event_id']]
feature_e = [x for x in data.columns if x not in ['flag', 'index', 'hit_id', 'event_id']]
data[feature_e] = scaler.fit_transform(data[feature_e])

evenidlist = list(set(data['event_id']))
grouplist = [data[data['event_id'] == i1] for i1 in evenidlist]
# print(data[data['hit_id']<0])
del data
gc.collect()

9473201
(13559712, 44)


0

In [8]:
timestep = 4000
trainpd = []
lenpd = []
for i1 in grouplist:
    if ~np.isnan(i1.iloc[0].flag):
        tnp = -np.ones((timestep,len(feature_e)))
        tnp[0:i1.shape[0],:] =  np.array(i1[feature_e])
        trainpd.append(tnp[np.newaxis,:])
#         trainpd.append(np.array(i1[feature_e]))
        lenpd.append(i1.shape[0])
trainpd = np.concatenate(trainpd, 0)
lenpd = np.array(lenpd)

labels = []
for i1 in grouplist:
    if ~np.isnan(i1.iloc[0].flag):
        tnp = np.zeros((timestep))
        tnp[0:i1.shape[0]] =  np.array(i1["flag"], dtype='int64')
        labels.append(tnp[np.newaxis,:])
#         labels.append(np.array(i1["flag"], dtype='int8'))
labels = np.concatenate(labels, 0)

testpd = []
testhitidpd = []
testeventidpd = []
cenum = 0
for i1 in grouplist:
    if np.isnan(i1.iloc[0].flag):
        cenum +=i1.shape[0] 
        tnp = -np.ones((timestep,len(feature_e)))
        thidp = -np.ones((timestep), dtype='int64')
        teidp = -np.ones((timestep), dtype='int64')
        tnp[0:i1.shape[0],:] =  np.array(i1[feature_e])
        testpd.append(tnp[np.newaxis,:])
        
        thidp[0:i1.shape[0]] =  np.array(i1["hit_id"], dtype='int64')
        testhitidpd.append(thidp[np.newaxis,:])
        teidp[0:i1.shape[0]] =  np.array(i1["event_id"], dtype='int64')
        testeventidpd.append(teidp[np.newaxis,:])
testpd = np.concatenate(testpd, 0)
testhitidpd = np.concatenate(testhitidpd, 0)
testeventidpd = np.concatenate(testeventidpd, 0)
print(cenum)
del grouplist
gc.collect()

4086511


8

In [9]:
print(trainpd.shape)
print(lenpd.shape)
print(labels.shape)
print(testpd.shape)

(9300, 4000, 41)
(9300,)
(9300, 4000)
(4015, 4000, 41)


In [18]:
# from sklearn.metrics import roc_curve, auc as auccc
keras.backend.clear_session()
def roc_auc_score_FIXED(y_true, y_pred):
    if len(np.unique(y_true)) == 1: # bug in roc_auc_score
#         return accuracy_score(y_true, np.rint(y_pred))
        return np.mean(accuracy_score(y_true, np.rint(y_pred)))
    y_true = y_true.reshape((-1))
    y_pred = y_pred.reshape((-1))
    return np.mean(roc_auc_score(y_true, y_pred))*1000

def auroc(y_true, y_pred):
    return tf.py_func( roc_auc_score_FIXED, (y_true, y_pred), tf.double)


# def mpcrossentropy(labellist, y_pred):
#     y_true,lenpd = labellist[0],int(labellist[1])
#     y_true = y_true[0:lenpd]
#     y_pred = y_pred[0:lenpd] 
#     return log_loss(y_true, y_pred)

# def mycrossentropy(labellist, y_pred):
#     y_true,lenpd = labellist[0],int(labellist[1])
#     loss1 = K.binary_crossentropy(y_true, y_pred)
#     return K.s(loss1) 

def mycrossentropy(y_true, y_pred):
    loss1 = K.binary_crossentropy(y_true, y_pred)
    return K.mean(loss1) 

def get_model_cnn():    
    embed_size = len(feature_e)
    inp = Input(shape=[None, embed_size])
#     x = Masking(mask_value=0.)(inp)
    x = inp
    filters = 30
    x = Conv1D(filters, 1, strides=(1), padding='same')(x)
    x = Activation('elu')(x)
    x = Dropout(0.5)(x)
#     filters = 15
#     x = Conv1D(filters, 1, strides=(1), padding='same')(inp)
#     x = Dense(60, activation='elu')(x)
    for iit in range(3):
        filters = 60*(1+iit)
        x = Conv1D(filters, 3, strides=(1), padding='same')(x)
        x = Dropout(0.5)(x)
        x = Activation('elu')(x)
    x = Dropout(0.5)(x)
    filters = 30
    x = Conv1D(filters, 1, strides=(1), padding='same')(x)
    x = Activation('elu')(x)
    filters = 10
    x = Conv1D(filters, 1, strides=(1), padding='same')(x)
    x = Activation('elu')(x)
    filters = 1
    x = Conv1D(filters, 1, strides=(1), padding='same')(x)
    x = Activation('sigmoid')(x)
#     x = Dense(1, activation='elu')(x)
    def sqqueeze(x):
        return K.squeeze(x, axis=-1)

    x = keras.layers.Lambda(sqqueeze)(x)
    
    model = Model(inputs=inp, outputs=x)
    model.compile(
        loss=mycrossentropy,
        optimizer='adam',
        metrics=[auroc])
    return model

def get_model_lstm():
    embed_size = len(feature_e)
    memn = 100

#     inp = Input(shape=[timestep, embed_size])
    inp = Input(shape=[None, embed_size])
    x = Masking(mask_value=-1.)(inp)
#     x = Bidirectional(LSTM(memn, return_sequences=True))(inp)
    x = Bidirectional(LSTM(memn, return_sequences=True))(x)
    x = Dense(memn, activation='relu')(x)
    x = Dropout(0.5)(x)
    x = Bidirectional(LSTM(memn // 2, return_sequences=True))(x)
#     x = Dense(10, activation='relu')(x)
    x = Dense(1, activation='sigmoid')(x)
    
    def sqqueeze(x):
        return K.squeeze(x, axis=-1)

    x = keras.layers.Lambda(sqqueeze)(x)
#     x = Flatten()(x)
    model = Model(inputs=inp, outputs=x)
    model.compile(
        loss=mycrossentropy,
#         loss='binary_crossentropy',
                  optimizer='adam',
#                   metrics=['accuracy'])
#                   metrics=[auc_roc])
                  metrics=[auroc])
#                   metrics=[auc])
#                   metrics=['accuracy', auroc])
    return model

pathf = os.path.join("..", "data", "particles")
model_path = os.path.join(pathf, "model", "tmp_cnn.h5")
model_path = os.path.join(pathf, "model", "tmp_lstm.h5")
log_path = os.path.join(pathf, "model")
checkpoint = ModelCheckpoint(model_path, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
tensorb = TensorBoard(log_dir=log_path, histogram_freq=10, write_graph=True, write_images=True,
                      embeddings_freq=0,
                      embeddings_layer_names=None, embeddings_metadata=None)
early = EarlyStopping(monitor="val_loss", mode="min", patience=200)
callbacks_list = [checkpoint, early, tensorb]  # early

if "lstm" in model_path:
    model = get_model_lstm()
else:
    model = get_model_cnn()
print(model.summary())

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, None, 41)          0         
_________________________________________________________________
masking_1 (Masking)          (None, None, 41)          0         
_________________________________________________________________
bidirectional_1 (Bidirection (None, None, 200)         113600    
_________________________________________________________________
dense_1 (Dense)              (None, None, 100)         20100     
_________________________________________________________________
dropout_1 (Dropout)          (None, None, 100)         0         
_________________________________________________________________
bidirectional_2 (Bidirection (None, None, 100)         60400     
_________________________________________________________________
dense_2 (Dense)              (None, None, 1)           101 

In [19]:
batch_size = 16
epochs = 100
model.load_weights(model_path)
start = time.time()
model.fit(trainpd, labels, batch_size=batch_size, epochs=epochs, validation_split=0.2, callbacks=callbacks_list)
end = time.time()
print("outbatch: " + str(end - start))

Train on 7440 samples, validate on 1860 samples




Epoch 1/100
7440/7440 [==============================] - 1588s 213ms/step - loss: 0.0158 - auroc: 999.5529 - val_loss: 0.0163 - val_auroc: 999.5581

Epoch 00001: val_loss improved from inf to 0.01627, saving model to ../data/particles/model/tmp_lstm.h5
Epoch 2/100
7440/7440 [==============================] - 1576s 212ms/step - loss: 0.0158 - auroc: 999.5563 - val_loss: 0.0163 - val_auroc: 999.5508

Epoch 00002: val_loss did not improve from 0.01627
Epoch 3/100
7440/7440 [==============================] - 1574s 212ms/step - loss: 0.0158 - auroc: 999.5540 - val_loss: 0.0162 - val_auroc: 999.5544

Epoch 00003: val_loss improved from 0.01627 to 0.01617, saving model to ../data/particles/model/tmp_lstm.h5
Epoch 4/100
7440/7440 [==============================] - 1576s 212ms/step - loss: 0.0158 - auroc: 999.5554 - val_loss: 0.0161 - val_auroc: 999.5613

Epoch 00004: val_loss improved from 0.01617 to 0.01614, saving model to ../data/particles

7440/7440 [==============================] - 1574s 212ms/step - loss: 0.0171 - auroc: 999.4653 - val_loss: 0.0167 - val_auroc: 999.5236

Epoch 00036: val_loss did not improve from 0.01602
Epoch 37/100
7440/7440 [==============================] - 1575s 212ms/step - loss: 0.0169 - auroc: 999.4788 - val_loss: 0.0167 - val_auroc: 999.5202

Epoch 00037: val_loss did not improve from 0.01602
Epoch 38/100
7440/7440 [==============================] - 1576s 212ms/step - loss: 0.0167 - auroc: 999.4866 - val_loss: 0.0166 - val_auroc: 999.5303

Epoch 00038: val_loss did not improve from 0.01602
Epoch 39/100
7440/7440 [==============================] - 1575s 212ms/step - loss: 0.0166 - auroc: 999.4968 - val_loss: 0.0165 - val_auroc: 999.5379

Epoch 00039: val_loss did not improve from 0.01602
Epoch 40/100
7440/7440 [==============================] - 1572s 211ms/step - loss: 0.0165 - auroc: 999.5017 - val_loss: 0.0165 - val_auroc: 999.5301

Epoch 00040: val_loss did not improve from 0.01602
Epoch 41

7440/7440 [==============================] - 1577s 212ms/step - loss: 0.0155 - auroc: 999.5628 - val_loss: 0.0162 - val_auroc: 999.5630

Epoch 00077: val_loss did not improve from 0.01602
Epoch 78/100
7440/7440 [==============================] - 1576s 212ms/step - loss: 0.0155 - auroc: 999.5639 - val_loss: 0.0161 - val_auroc: 999.5530

Epoch 00078: val_loss did not improve from 0.01602
Epoch 79/100
7440/7440 [==============================] - 1577s 212ms/step - loss: 0.0155 - auroc: 999.5658 - val_loss: 0.0161 - val_auroc: 999.5678

Epoch 00079: val_loss did not improve from 0.01602
Epoch 80/100
7440/7440 [==============================] - 1576s 212ms/step - loss: 0.0155 - auroc: 999.5644 - val_loss: 0.0162 - val_auroc: 999.5616

Epoch 00080: val_loss did not improve from 0.01602
Epoch 81/100
7440/7440 [==============================] - 1579s 212ms/step - loss: 0.0156 - auroc: 999.5641 - val_loss: 0.0160 - val_auroc: 999.5713

Epoch 00081: val_loss did not improve from 0.01602
Epoch 82

In [16]:
# model_path = os.path.join(pathf, "model", "tmp_lstm_1.h5")
model.load_weights(model_path)
fy_submission = model.predict(testpd)

In [17]:
#阈值大概在0.2-0.4之间 本题对召回率较敏感，可适当降低一下阈值
fy_submission=fy_submission.reshape((-1))
testhitidpd=testhitidpd.reshape((-1))
testeventidpd=testeventidpd.reshape((-1))
# print(set(testeventidpd))
# 1. 
thre = 0.3
sub = pd.DataFrame()
sub['hit_id'] = testhitidpd
sub['flag_pred'] = fy_submission
sub['event_id'] = testeventidpd
sub['flag_pred'] = sub['flag_pred'].apply(lambda x: 1 if x >= thre else 0)
sub=sub[sub['event_id']!=-1]

# 2. group 为正的比率小于阈值，全置零。
threshe = 0.15
def filtfun(data):
    uper = sum(data['flag_pred'])
    anum =data.shape[0]
    return uper/data.shape[0]

filt = pd.DataFrame()
filt["event_id"] = sub.groupby(["event_id"])["event_id"].mean()
filt['ratioss'] = sub.groupby(['event_id']).apply(filtfun)
filt.reset_index(drop=True, inplace=True)
sub = pd.merge(sub, filt, on='event_id', how='left')
sub.loc[sub['ratioss'] < threshe,"flag_pred"] = 0
sub=sub[["hit_id","flag_pred","event_id"]]
sub.to_csv(os.path.join(pathf, "subsample.csv").format(sub['flag_pred'].mean()), index=False)
print(sub.head())

   hit_id  flag_pred  event_id
0     223          0         9
1     243          0         9
2     382          0         9
3     159          0         9
4     403          0         9
